In [ ]:
#import libraries
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.utils import normalize, to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array


In [2]:
# Define dataset path
image_directory = 'datasets/'
no_tumor_images = os.listdir(image_directory + 'no/')
yes_tumor_images = os.listdir(image_directory + 'yes/')


In [3]:
# Preprocess dataset
dataset = []
label = []
INPUT_SIZE = 224  # Adjusted for transfer learning compatibility


In [4]:
# Load images function
def load_images(image_list, label_value, folder):
    for image_name in image_list:
        if image_name.endswith('.jpg'):
            image_path = os.path.join(image_directory, folder, image_name)
            image = cv2.imread(image_path)
            if image is not None:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (INPUT_SIZE, INPUT_SIZE))
                dataset.append(img_to_array(image))
                label.append(label_value)



In [5]:
# Load images
load_images(no_tumor_images, 0, 'no')
load_images(yes_tumor_images, 1, 'yes')


In [6]:
# Convert to NumPy arrays
dataset = np.array(dataset, dtype=np.float32) / 255.0  # Normalize
label = np.array(label)


In [7]:

# Split data
x_train, x_test, y_train, y_test = train_test_split(dataset, label, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)


In [9]:
# Build CNN model
cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(INPUT_SIZE, INPUT_SIZE, 3)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
cnn_model.fit(x_train, y_train, batch_size=16, epochs=10, validation_data=(x_test, y_test))
cnn_model.save('BrainTumor_CNN.h5')


Epoch 1/10
150/150 [==============================] - 306s 2s/step - loss: 0.6087 - accuracy: 0.8208 - val_loss: 0.1810 - val_accuracy: 0.9500
Epoch 2/10
150/150 [==============================] - 284s 2s/step - loss: 0.1434 - accuracy: 0.9550 - val_loss: 0.1050 - val_accuracy: 0.9717
Epoch 3/10
150/150 [==============================] - 294s 2s/step - loss: 0.0807 - accuracy: 0.9746 - val_loss: 0.1302 - val_accuracy: 0.9567
Epoch 4/10
150/150 [==============================] - 244s 2s/step - loss: 0.0541 - accuracy: 0.9867 - val_loss: 0.0852 - val_accuracy: 0.9767
Epoch 5/10
150/150 [==============================] - 233s 2s/step - loss: 0.0180 - accuracy: 0.9962 - val_loss: 0.1003 - val_accuracy: 0.9783
Epoch 6/10
150/150 [==============================] - 230s 2s/step - loss: 0.0138 - accuracy: 0.9971 - val_loss: 0.0871 - val_accuracy: 0.9867
Epoch 7/10
150/150 [==============================] - 223s 1s/step - loss: 0.0107 - accuracy: 0.9971 - val_loss: 0.1187 - val_accuracy: 0.9800

In [11]:

# Build Transfer Learning Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(INPUT_SIZE, INPUT_SIZE, 3))
for layer in base_model.layers:
    layer.trainable = False

transfer_model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

transfer_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
transfer_model.fit(x_train, y_train, batch_size=16, epochs=10, validation_data=(x_test, y_test))
transfer_model.save('BrainTumor_TransferLearning.h5')


Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5: None -- [WinError 10054] An existing connection was forcibly closed by the remote host